In [1]:
import requests
import json
import pandas as pd
import numpy as np
import datetime


In [2]:
from test_key import g_key
import gmaps
import os
import matplotlib.pyplot as plt
from test_key import bls_api_key


## Demographic unemployment rate

In [3]:
# Pull gender, race, educational attainment, and age data from Bureau of Labor Statistics

url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
headers = {"content-type": "application/json"}

target_series = ["LNS14000000", "LNS14000001", "LNS14000002", "LNS14000003", "LNS14000006", "LNS14032183", "LNS14000009",
         "LNS14027659", "LNS14027660", "LNS14027689", "LNS14027662", "LNS14024887", "LNS14000089", "LNS14000091",
                "LNS14000093", "LNS14024230"]

start_year = "2012"
end_year = "2018"

parameters = json.dumps({"seriesid": target_series,
             "startyear": start_year,
             "endyear": end_year,
             "registrationkey": bls_api_key})

p = requests.post(url, data=parameters, headers=headers)

json_data = json.loads(p.text)

json_data

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 688,
 'message': [],
 'Results': {'series': [{'seriesID': 'LNS14000000',
    'data': [{'year': '2018',
      'period': 'M12',
      'periodName': 'December',
      'value': '3.9',
      'footnotes': [{}]},
     {'year': '2018',
      'period': 'M11',
      'periodName': 'November',
      'value': '3.7',
      'footnotes': [{}]},
     {'year': '2018',
      'period': 'M10',
      'periodName': 'October',
      'value': '3.8',
      'footnotes': [{}]},
     {'year': '2018',
      'period': 'M09',
      'periodName': 'September',
      'value': '3.7',
      'footnotes': [{}]},
     {'year': '2018',
      'period': 'M08',
      'periodName': 'August',
      'value': '3.8',
      'footnotes': [{}]},
     {'year': '2018',
      'period': 'M07',
      'periodName': 'July',
      'value': '3.8',
      'footnotes': [{}]},
     {'year': '2018',
      'period': 'M06',
      'periodName': 'June',
      'value': '4.0',
      'footnotes': [{}]},
     

In [4]:
years = []
months = []
date = []

for data_point in np.arange(len(json_data["Results"]["series"][0]["data"])):
        years.append(json_data["Results"]["series"][0]["data"][data_point]["year"])
        months.append(json_data["Results"]["series"][0]["data"][data_point]["periodName"])
        date.append(f"{json_data['Results']['series'][0]['data'][data_point]['year']}-{json_data['Results']['series'][0]['data'][data_point]['period'][1:]}")

In [5]:
# Create DataFrame

unemployment_df = pd.DataFrame({"Date": date,
                                "Year": years,
                                "Month": months,
                                "LNS14000000" : "",
                                "LNS14000001" : "",
                                "LNS14000002" : "",
                                "LNS14000003" : "",
                                "LNS14000006" : "",
                                "LNS14032183" : "",
                                "LNS14000009" : "",
                                "LNS14027659" : "",
                                "LNS14027660" : "",
                                "LNS14027689" : "",
                                "LNS14027662" : "",
                                "LNS14024887" : "",
                                "LNS14000089" : "",
                                "LNS14000091" : "",
                                "LNS14000093" : "",
                                "LNS14024230" : ""})
unemployment_df.head()

,Date,Year,Month,LNS14000000,LNS14000001,LNS14000002,LNS14000003,LNS14000006,LNS14032183,LNS14000009,LNS14027659,LNS14027660,LNS14027689,LNS14027662,LNS14024887,LNS14000089,LNS14000091,LNS14000093,LNS14024230
0,2018-12,2018,December,,,,,,,,,,,,,,,,
1,2018-11,2018,November,,,,,,,,,,,,,,,,
2,2018-10,2018,October,,,,,,,,,,,,,,,,
3,2018-09,2018,September,,,,,,,,,,,,,,,,
4,2018-08,2018,August,,,,,,,,,,,,,,,,


In [6]:
# Fills in values

for series in np.arange(len(json_data["Results"]["series"])):
    for data_point in np.arange(len(json_data["Results"]["series"][series]["data"])):
        unemployment_df.loc[data_point, json_data["Results"]["series"][series]["seriesID"]] = json_data["Results"]["series"][series]["data"][data_point]["value"]

In [7]:
# Rename column labels

unemployment_df.rename(columns={"LNS14000000": "Unemployment Rate for 16yo and over",
                                "LNS14000001": "Unemployment Rate for Men 16yo and over",
                                "LNS14000002": "Unemployment Rate for Women 16yo and over",
                                "LNS14000003": "Unemployment Rate for White people 16yo and over",
                                "LNS14000006": "Unemployment Rate for African Americans 16yo and over",
                                "LNS14032183": "Unemployment Rate for Asian people 16yo and over",
                                "LNS14000009": "Unemployment Rate for Hispanic people 16yo and over",
                                "LNS14027659": "Unemployment Rate for 25yo and older with less than High School Diploma",
                                "LNS14027660": "Unemployment Rate for 25yo and older with High School Degree",
                                "LNS14027689": "Unemployment Rate for 25yo and older with some college",
                                "LNS14027662": "Unemployment Rate for 25yo and older with Bachelor's degree or higher",
                                "LNS14024887": "Unemployment Rate for 16-24yo",
                                "LNS14000089": "Unemployment Rate for 25-34yo",
                                "LNS14000091": "Unemployment Rate for 35-44yo",
                                "LNS14000093": "Unemployment Rate for 45-54yo",
                                "LNS14024230": "Unemployment Rate for 55yo and over"},
                      inplace=True)

unemployment_df.sort_values(by="Date", ascending=True, inplace=True)
unemployment_df.to_csv("Unemployment_Rates.csv", index=False)

unemployment_df

,Date,Year,Month,Unemployment Rate for 16yo and over,Unemployment Rate for Men 16yo and over,Unemployment Rate for Women 16yo and over,Unemployment Rate for White people 16yo and over,Unemployment Rate for African Americans 16yo and over,Unemployment Rate for Asian people 16yo and over,Unemployment Rate for Hispanic people 16yo and over,Unemployment Rate for 25yo and older with less than High School Diploma,Unemployment Rate for 25yo and older with High School Degree,Unemployment Rate for 25yo and older with some college,Unemployment Rate for 25yo and older with Bachelor's degree or higher,Unemployment Rate for 16-24yo,Unemployment Rate for 25-34yo,Unemployment Rate for 35-44yo,Unemployment Rate for 45-54yo,Unemployment Rate for 55yo and over
83,2012-01,2012,January,8.3,8.4,8.2,7.4,13.6,6.7,10.7,13.0,8.5,7.1,4.3,16.1,9.0,6.8,6.4,5.9
82,2012-02,2012,February,8.3,8.4,8.1,7.4,14.0,6.2,10.9,13.1,8.3,7.2,4.2,16.5,8.6,6.8,6.5,5.8
81,2012-03,2012,March,8.2,8.4,8.0,7.3,14.0,6.4,10.6,12.8,8.1,7.5,4.1,16.2,8.7,6.4,6.3,6.2
80,2012-04,2012,April,8.2,8.3,8.0,7.4,13.3,5.4,10.3,12.5,8.0,7.6,4.0,16.6,8.2,6.6,6.2,6.3
79,2012-05,2012,May,8.2,8.4,7.9,7.4,13.5,5.5,10.9,12.9,8.3,7.7,3.9,16.1,8.2,6.7,6.3,6.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2018-08,2018,August,3.8,3.8,3.8,3.4,6.3,3.0,4.7,5.7,3.9,3.5,2.0,8.5,4.1,2.8,2.5,3.0
3,2018-09,2018,September,3.7,3.8,3.7,3.3,6.2,3.5,4.6,5.7,3.8,3.2,2.0,8.6,3.6,3.0,2.6,2.8
2,2018-10,2018,October,3.8,3.8,3.8,3.3,6.3,3.1,4.4,5.9,4.0,3.1,2.0,8.4,3.7,3.0,2.8,2.8
1,2018-11,2018,November,3.7,3.6,3.8,3.4,6.1,2.8,4.5,5.6,3.5,3.1,2.2,8.2,3.7,2.9,2.7,2.8


## Statewide unemployment rate

In [8]:
# Pull state data from Bureau of Labor Statistics

url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
headers = {"content-type": "application/json"}

target_series = ["LASST010000000000003", "LASST020000000000003", "LASST040000000000003", "LASST050000000000003",
                 "LASST060000000000003", "LASST080000000000003", "LASST090000000000003", "LASST100000000000003",
                 "LASST110000000000003", "LASST120000000000003", "LASST130000000000003", "LASST150000000000003",
                 "LASST160000000000003", "LASST170000000000003", "LASST180000000000003", "LASST190000000000003",
                 "LASST200000000000003", "LASST210000000000003", "LASST220000000000003", "LASST230000000000003",
                 "LASST240000000000003", "LASST250000000000003", "LASST260000000000003", "LASST270000000000003",
                 "LASST280000000000003", "LASST290000000000003", "LASST300000000000003", "LASST310000000000003",
                 "LASST320000000000003", "LASST330000000000003", "LASST340000000000003", "LASST350000000000003",
                 "LASST360000000000003", "LASST370000000000003", "LASST380000000000003", "LASST390000000000003",
                 "LASST400000000000003", "LASST410000000000003", "LASST420000000000003", "LASST720000000000003",
                 "LASST440000000000003", "LASST450000000000003", "LASST460000000000003", "LASST470000000000003",
                 "LASST480000000000003", "LASST490000000000003", "LASST500000000000003", "LASST510000000000003",
                 "LASST530000000000003", "LASST540000000000003"]




parameters = json.dumps({"seriesid": target_series,
             "startyear": start_year,
             "endyear": end_year,
             "registrationkey": bls_api_key})

p = requests.post(url, data=parameters, headers=headers)
json_data2 = json.loads(p.text)


json_data2

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 2029,
 'message': [],
 'Results': {'series': [{'seriesID': 'LASST010000000000003',
    'data': [{'year': '2018',
      'period': 'M12',
      'periodName': 'December',
      'value': '3.6',
      'footnotes': [{'code': 'R',
        'text': 'Data were subject to revision on March 4, 2020.'}]},
     {'year': '2018',
      'period': 'M11',
      'periodName': 'November',
      'value': '3.7',
      'footnotes': [{'code': 'R',
        'text': 'Data were subject to revision on March 4, 2020.'}]},
     {'year': '2018',
      'period': 'M10',
      'periodName': 'October',
      'value': '3.7',
      'footnotes': [{'code': 'R',
        'text': 'Data were subject to revision on March 4, 2020.'}]},
     {'year': '2018',
      'period': 'M09',
      'periodName': 'September',
      'value': '3.8',
      'footnotes': [{'code': 'R',
        'text': 'Data were subject to revision on March 4, 2020.'}]},
     {'year': '2018',
      'period': 'M08',
   

In [9]:
# Pull state data from Bureau of Labor Statistics

url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
headers = {"content-type": "application/json"}

target_series = ["LASST550000000000003", "LASST560000000000003"]




parameters = json.dumps({"seriesid": target_series,
             "startyear": start_year,
             "endyear": end_year,
             "registrationkey": bls_api_key})

p = requests.post(url, data=parameters, headers=headers)

json_data3 = json.loads(p.text)

json_data3

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 201,
 'message': [],
 'Results': {'series': [{'seriesID': 'LASST550000000000003',
    'data': [{'year': '2018',
      'period': 'M12',
      'periodName': 'December',
      'value': '3.0',
      'footnotes': [{'code': 'R',
        'text': 'Data were subject to revision on March 4, 2020.'}]},
     {'year': '2018',
      'period': 'M11',
      'periodName': 'November',
      'value': '3.0',
      'footnotes': [{'code': 'R',
        'text': 'Data were subject to revision on March 4, 2020.'}]},
     {'year': '2018',
      'period': 'M10',
      'periodName': 'October',
      'value': '3.0',
      'footnotes': [{'code': 'R',
        'text': 'Data were subject to revision on March 4, 2020.'}]},
     {'year': '2018',
      'period': 'M09',
      'periodName': 'September',
      'value': '3.0',
      'footnotes': [{'code': 'R',
        'text': 'Data were subject to revision on March 4, 2020.'}]},
     {'year': '2018',
      'period': 'M08',
    

In [10]:
years = []
months = []
date = []

for data_point in np.arange(len(json_data2["Results"]["series"][0]["data"])):
        years.append(json_data2["Results"]["series"][0]["data"][data_point]["year"])
        months.append(json_data2["Results"]["series"][0]["data"][data_point]["periodName"])
        date.append(f"{json_data['Results']['series'][0]['data'][data_point]['year']}-{json_data['Results']['series'][0]['data'][data_point]['period'][1:]}")

In [11]:
# Create DataFrame

state_unemployment_df = pd.DataFrame({"Date": date,
                                "Year": years,
                                "Month": months,
                                "LASST010000000000003": "", "LASST020000000000003": "", "LASST040000000000003": "",
                                "LASST050000000000003": "", "LASST060000000000003": "", "LASST080000000000003": "",
                                "LASST090000000000003": "", "LASST100000000000003": "", "LASST110000000000003": "",
                                "LASST120000000000003": "", "LASST130000000000003": "", "LASST150000000000003": "",
                                "LASST160000000000003": "", "LASST170000000000003": "", "LASST180000000000003": "",
                                "LASST190000000000003": "", "LASST200000000000003": "", "LASST210000000000003": "",
                                "LASST220000000000003": "", "LASST230000000000003": "", "LASST240000000000003": "",
                                "LASST250000000000003": "", "LASST260000000000003": "", "LASST270000000000003": "",
                                "LASST280000000000003": "", "LASST290000000000003": "", "LASST300000000000003": "",
                                "LASST310000000000003": "", "LASST320000000000003": "", "LASST330000000000003": "",
                                "LASST340000000000003": "", "LASST350000000000003": "", "LASST360000000000003": "",
                                "LASST370000000000003": "", "LASST380000000000003": "", "LASST390000000000003": "",
                                "LASST400000000000003": "", "LASST410000000000003": "", "LASST420000000000003": "",
                                "LASST720000000000003": "", "LASST440000000000003": "", "LASST450000000000003": "",
                                "LASST460000000000003": "", "LASST470000000000003": "", "LASST480000000000003": "",
                                "LASST490000000000003": "", "LASST500000000000003": "", "LASST510000000000003": "",
                                "LASST530000000000003": "", "LASST540000000000003": "", "LASST550000000000003": "",
                                "LASST560000000000003": ""})
state_unemployment_df
state_unemployment_df

,Date,Year,Month,LASST010000000000003,LASST020000000000003,LASST040000000000003,LASST050000000000003,LASST060000000000003,LASST080000000000003,LASST090000000000003,...,LASST460000000000003,LASST470000000000003,LASST480000000000003,LASST490000000000003,LASST500000000000003,LASST510000000000003,LASST530000000000003,LASST540000000000003,LASST550000000000003,LASST560000000000003
0,2018-12,2018,December,,,,,,,,...,,,,,,,,,,
1,2018-11,2018,November,,,,,,,,...,,,,,,,,,,
2,2018-10,2018,October,,,,,,,,...,,,,,,,,,,
3,2018-09,2018,September,,,,,,,,...,,,,,,,,,,
4,2018-08,2018,August,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,2012-05,2012,May,,,,,,,,...,,,,,,,,,,
80,2012-04,2012,April,,,,,,,,...,,,,,,,,,,
81,2012-03,2012,March,,,,,,,,...,,,,,,,,,,
82,2012-02,2012,February,,,,,,,,...,,,,,,,,,,


In [12]:

state_unemployment_df = pd.DataFrame({"Date": date,
                                      "Year": years,
                                      "Month": months,
                                      
                                "LASST010000000000003": "", "LASST020000000000003": "", "LASST040000000000003": "",
                                "LASST050000000000003": "", "LASST060000000000003": "", "LASST080000000000003": "",
                                "LASST090000000000003": "", "LASST100000000000003": "", "LASST110000000000003": "",
                                "LASST120000000000003": "", "LASST130000000000003": "", "LASST150000000000003": "",
                                "LASST160000000000003": "", "LASST170000000000003": "", "LASST180000000000003": "",
                                "LASST190000000000003": "", "LASST200000000000003": "", "LASST210000000000003": "",
                                "LASST220000000000003": "", "LASST230000000000003": "", "LASST240000000000003": "",
                                "LASST250000000000003": "", "LASST260000000000003": "", "LASST270000000000003": "",
                                "LASST280000000000003": "", "LASST290000000000003": "", "LASST300000000000003": "",
                                "LASST310000000000003": "", "LASST320000000000003": "", "LASST330000000000003": "",
                                "LASST340000000000003": "", "LASST350000000000003": "", "LASST360000000000003": "",
                                "LASST370000000000003": "", "LASST380000000000003": "", "LASST390000000000003": "",
                                "LASST400000000000003": "", "LASST410000000000003": "", "LASST420000000000003": "",
                                "LASST720000000000003": "", "LASST440000000000003": "", "LASST450000000000003": "",
                                "LASST460000000000003": "", "LASST470000000000003": "", "LASST480000000000003": "",
                                "LASST490000000000003": "", "LASST500000000000003": "", "LASST510000000000003": "",
                                "LASST530000000000003": "", "LASST540000000000003": "", "LASST550000000000003": "",
                                "LASST560000000000003": ""})
state_unemployment_df.head()

,Date,Year,Month,LASST010000000000003,LASST020000000000003,LASST040000000000003,LASST050000000000003,LASST060000000000003,LASST080000000000003,LASST090000000000003,...,LASST460000000000003,LASST470000000000003,LASST480000000000003,LASST490000000000003,LASST500000000000003,LASST510000000000003,LASST530000000000003,LASST540000000000003,LASST550000000000003,LASST560000000000003
0,2018-12,2018,December,,,,,,,,...,,,,,,,,,,
1,2018-11,2018,November,,,,,,,,...,,,,,,,,,,
2,2018-10,2018,October,,,,,,,,...,,,,,,,,,,
3,2018-09,2018,September,,,,,,,,...,,,,,,,,,,
4,2018-08,2018,August,,,,,,,,...,,,,,,,,,,


In [13]:
# Fills in values

for series2 in np.arange(len(json_data2["Results"]["series"])):
    for data_point2 in np.arange(len(json_data2["Results"]["series"][series2]["data"])):
        state_unemployment_df.loc[data_point2, json_data2["Results"]["series"][series2]["seriesID"]] = json_data2["Results"]["series"][series2]["data"][data_point2]["value"]

for series3 in np.arange(len(json_data3["Results"]["series"])):
    for data_point3 in np.arange(len(json_data3["Results"]["series"][series3]["data"])):
        state_unemployment_df.loc[data_point3, json_data3["Results"]["series"][series3]["seriesID"]] = json_data3["Results"]["series"][series3]["data"][data_point3]["value"]

In [14]:
# Rename Column labels

state_unemployment_df.rename(columns={"LASST010000000000003" : "Alabama", "LASST020000000000003" : "Alaska",
                                      "LASST040000000000003" : "Arizona", "LASST050000000000003" : "Arkansas",
                                      "LASST060000000000003" : "California", "LASST080000000000003" : "Colorado",
                                      "LASST090000000000003" : "Connecticut", "LASST100000000000003" : "Delaware",
                                      "LASST110000000000003" : "District Of Columbia", "LASST120000000000003" : "Florida",
                                      "LASST130000000000003" : "Georgia", "LASST150000000000003" : "Hawaii",
                                      "LASST160000000000003" : "Idaho", "LASST170000000000003" : "Illinois",
                                      "LASST180000000000003" : "Indiana", "LASST190000000000003" : "Iowa",
                                      "LASST200000000000003" : "Kansas", "LASST210000000000003" : "Kentucky",
                                      "LASST220000000000003" : "Louisiana", "LASST230000000000003" : "Maine",
                                      "LASST240000000000003" : "Maryland", "LASST250000000000003" : "Massachusetts",
                                      "LASST260000000000003" : "Michigan", "LASST270000000000003" : "Minnesota",
                                      "LASST280000000000003" : "Mississippi", "LASST290000000000003" : "Missouri",
                                      "LASST300000000000003" : "Montana", "LASST310000000000003" : "Nebraska",
                                      "LASST320000000000003" : "Nevada", "LASST330000000000003" : "New Hampshire",
                                      "LASST340000000000003" : "New Jersey", "LASST350000000000003" : "New Mexico",
                                      "LASST360000000000003" : "New York", "LASST370000000000003" : "North Carolina",
                                      "LASST380000000000003" : "North Dakota", "LASST390000000000003" : "Ohio",
                                      "LASST400000000000003" : "Oklahoma", "LASST410000000000003" : "Oregon",
                                      "LASST420000000000003" : "Pennsylvania", "LASST720000000000003" : "Puerto Rico",
                                      "LASST440000000000003" : "Rhode Island", "LASST450000000000003" : "South Carolina",
                                      "LASST460000000000003" : "South Dakota", "LASST470000000000003" : "Tennessee",
                                      "LASST480000000000003" : "Texas", "LASST490000000000003" : "Utah",
                                      "LASST500000000000003" : "Vermont", "LASST510000000000003" : "Virginia",
                                      "LASST530000000000003" : "Washington", "LASST540000000000003" : "West Virginia",
                                      "LASST550000000000003" : "Wisconsin", "LASST560000000000003" : "Wyoming"},
                             inplace=True)

state_unemployment_df.sort_values(by="Date", ascending=True, inplace=True)

state_unemployment_df.to_csv("State_Unemployment_Rates.csv", index=False)

state_unemployment_df

,Date,Year,Month,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
83,2012-01,2012,January,8.1,7.4,8.7,7.7,11.0,8.2,8.2,...,4.4,7.9,7.1,5.9,5.1,6.3,8.6,7.4,7.2,5.5
82,2012-02,2012,February,7.9,7.3,8.7,7.6,10.9,8.1,8.2,...,4.3,7.8,7.0,5.8,5.0,6.2,8.5,7.3,7.1,5.4
81,2012-03,2012,March,8.0,7.3,8.6,7.6,10.8,8.1,8.3,...,4.3,7.8,6.9,5.7,5.0,6.2,8.5,7.3,7.1,5.3
80,2012-04,2012,April,8.1,7.2,8.6,7.6,10.7,8.1,8.3,...,4.3,7.9,6.9,5.6,5.0,6.2,8.4,7.4,7.1,5.3
79,2012-05,2012,May,8.2,7.2,8.5,7.6,10.6,8.0,8.4,...,4.3,7.9,6.9,5.5,5.0,6.2,8.4,7.5,7.1,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2018-08,2018,August,3.9,6.4,4.7,3.6,4.2,3.2,4.0,...,3.0,3.5,3.7,3.0,2.5,2.9,4.4,5.1,3.0,3.9
3,2018-09,2018,September,3.8,6.3,4.7,3.6,4.2,3.3,4.0,...,3.0,3.5,3.7,3.0,2.5,2.9,4.4,5.1,3.0,3.9
2,2018-10,2018,October,3.7,6.3,4.8,3.6,4.2,3.3,3.9,...,3.0,3.4,3.7,3.0,2.4,2.9,4.4,5.0,3.0,3.8
1,2018-11,2018,November,3.7,6.2,4.8,3.6,4.2,3.3,3.9,...,3.0,3.4,3.7,3.0,2.4,2.9,4.5,5.0,3.0,3.7


In [ ]:
# df[['A', 'C']] = df[['A', 'C']].apply(pd.to_numeric) 
# print(df.dtypes) 

In [15]:
states_df = pd.read_csv("states.csv", delimiter="\t")
states_df=states_df.loc[states_df["name"]!="District of Columbia"]
state_names = states_df["name"]
state_unemployment_df[state_names] = state_unemployment_df[state_names].apply(pd.to_numeric)

In [16]:
state_unemployment_df.dtypes

Date                     object
Year                     object
Month                    object
Alabama                 float64
Alaska                  float64
Arizona                 float64
Arkansas                float64
California              float64
Colorado                float64
Connecticut             float64
Delaware                float64
District Of Columbia     object
Florida                 float64
Georgia                 float64
Hawaii                  float64
Idaho                   float64
Illinois                float64
Indiana                 float64
Iowa                    float64
Kansas                  float64
Kentucky                float64
Louisiana               float64
Maine                   float64
Maryland                float64
Massachusetts           float64
Michigan                float64
Minnesota               float64
Mississippi             float64
Missouri                float64
Montana                 float64
Nebraska                float64
Nevada  

In [17]:
state_unemployment_df_2 = state_unemployment_df.drop(["Date","Month"] , axis=1)
state_unemployment_df_2 = state_unemployment_df_2.groupby("Year").mean()
state_unemployment_df_2 = state_unemployment_df_2.transpose()
state_unemployment_df_2.head()


Year,2012,2013,2014,2015,2016,2017,2018
Alabama,7.983333,7.216667,6.758333,6.083333,5.841667,4.416667,3.875000
Alaska,7.133333,7.000000,6.866667,6.533333,6.858333,6.883333,6.450000
Arizona,8.333333,7.741667,6.800000,6.058333,5.375000,4.858333,4.741667
Arkansas,7.575000,7.216667,6.016667,5.008333,3.966667,3.683333,3.641667
California,10.375000,8.941667,7.500000,6.208333,5.491667,4.791667,4.258333


In [18]:
#df.rename(columns={"A": "a", "B": "c"})
state_unemployment_df_2.reset_index(inplace=True)
state_unemployment_df_2=state_unemployment_df_2.rename(columns={"index":"State"})
state_unemployment_df_2

Year,State,2012,2013,2014,2015,2016,2017,2018
0,Alabama,7.983333,7.216667,6.758333,6.083333,5.841667,4.416667,3.875000
1,Alaska,7.133333,7.000000,6.866667,6.533333,6.858333,6.883333,6.450000
2,Arizona,8.333333,7.741667,6.800000,6.058333,5.375000,4.858333,4.741667
3,Arkansas,7.575000,7.216667,6.016667,5.008333,3.966667,3.683333,3.641667
4,California,10.375000,8.941667,7.500000,6.208333,5.491667,4.791667,4.258333
5,Colorado,7.891667,6.850000,4.991667,3.883333,3.300000,2.791667,3.158333
6,Connecticut,8.350000,7.808333,6.625000,5.725000,5.166667,4.700000,4.158333
7,Delaware,7.208333,6.691667,5.716667,4.858333,4.525000,4.466667,3.783333
8,Florida,8.483333,7.233333,6.275000,5.450000,4.816667,4.225000,3.591667
9,Georgia,9.191667,8.200000,7.100000,5.983333,5.383333,4.691667,3.950000


In [19]:
state_unemployment_df_2.columns

Index(['State', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], dtype='object', name='Year')

In [20]:
states_df.columns

Index(['state', 'latitude', 'longitude', 'name'], dtype='object')

In [22]:
state_unemployment_df_2.set_index("State", inplace=True)

In [ ]:
#pd.merge(frame_1, frame_2, how='left', left_on='county_ID', right_on='countyid')
# state_unemployment_df_2 = pd.merge(state_unemployment_df_2, states_df, right_on="name",left_on="State", how="inner")
# state_unemployment_df_2= state_unemployment_df_2.drop(["state"],axis =1)
# state_unemployment_df_2.to_csv("state_unemployment_rate_1_csv")
# state_unemployment_df_2.head()

In [25]:

state_1218_df = state_unemployment_df_2.T
#for state in state_unemployment_df_2["State"]:

state_1218_df.head(15)   

State,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Florida,Georgia,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
Year,,,,,,,,,,,,,,,,,,,,,
2012,7.983333,7.133333,8.333333,7.575000,10.375000,7.891667,8.350000,7.208333,8.483333,9.191667,...,4.266667,7.825000,6.741667,5.400000,4.925000,6.058333,8.108333,7.491667,7.033333,5.308333
2013,7.216667,7.000000,7.741667,7.216667,8.941667,6.850000,7.808333,6.691667,7.233333,8.200000,...,3.783333,7.733333,6.250000,4.591667,4.408333,5.700000,7.033333,6.791667,6.700000,4.716667
2014,6.758333,6.866667,6.800000,6.016667,7.500000,4.991667,6.625000,5.716667,6.275000,7.100000,...,3.408333,6.600000,5.116667,3.816667,3.933333,5.233333,6.125000,6.633333,5.400000,4.133333
2015,6.083333,6.533333,6.058333,5.008333,6.208333,3.883333,5.725000,4.858333,5.450000,5.983333,...,3.100000,5.600000,4.425000,3.600000,3.558333,4.458333,5.641667,6.733333,4.525000,4.333333
2016,5.841667,6.858333,5.375000,3.966667,5.491667,3.300000,5.166667,4.525000,4.816667,5.383333,...,2.983333,4.733333,4.633333,3.450000,3.216667,4.100000,5.258333,6.075000,4.008333,5.316667
2017,4.416667,6.883333,4.858333,3.683333,4.791667,2.791667,4.700000,4.466667,4.225000,4.691667,...,3.208333,3.766667,4.308333,3.258333,2.941667,3.700000,4.708333,5.191667,3.266667,4.166667
2018,3.875000,6.450000,4.741667,3.641667,4.258333,3.158333,4.158333,3.783333,3.591667,3.950000,...,3.075000,3.525000,3.841667,3.050000,2.550000,3.041667,4.458333,5.208333,3.000000,3.866667


In [28]:
for col in state_1218_df.columns[1:]:
    print(col , state_1218_df[col].astype(float).mean())

Alaska 6.817857142857143
Arizona 6.272619047619048
Arkansas 5.301190476190476
California 6.795238095238096
Colorado 4.6952380952380945
Connecticut 6.076190476190477
Delaware 5.321428571428571
Florida 5.7250000000000005
Georgia 6.357142857142856
Hawaii 3.8166666666666664
Idaho 4.5928571428571425
Illinois 6.595238095238095
Indiana 5.463095238095237
Iowa 3.875
Kansas 4.389285714285713
Kentucky 6.038095238095238
Louisiana 6.0738095238095235
Maine 4.927380952380952
Maryland 5.292857142857143
Massachusetts 4.985714285714286
Michigan 6.328571428571429
Minnesota 4.089285714285714
Mississippi 6.736904761904762
Missouri 5.188095238095237
Montana 4.569047619047619
Nebraska 3.279761904761905
Nevada 7.2369047619047615
New Hampshire 3.7702380952380947
New Jersey 6.254761904761905
New Mexico 6.3690476190476195
New York 5.932142857142858
North Carolina 6.113095238095239
North Dakota 2.826190476190476
Ohio 5.7178571428571425
Oklahoma 4.559523809523809
Oregon 6.003571428571427
Pennsylvania 5.83095238095

In [27]:
# avg_1218_df= for col in state_1218_df.columns[1:]:
#     print(col , state_1218_df[col].astype(float).mean())
# avg_1218_df.to_csv("Resources/state_avg_1218.csv")

SyntaxError: invalid syntax (<ipython-input-27-f8cf3017ce6d>, line 1)

In [34]:
import pandas as pd
avg_file_2="Resources/state_avg_1218.csv"
avg_file_2_df=pd.read_csv(avg_file_2)
avg_file_2_df.head(10)

,State,avg1218
0,Alaska,6.817857
1,Arizona,6.272619
2,Arkansas,5.301190
3,California,6.795238
4,Colorado,4.695238
5,Connecticut,6.076190
6,Delaware,5.321429
7,Florida,5.725000
8,Georgia,6.357143
9,Hawaii,3.816667


In [36]:
avg_file_2_df.rename(columns={"State" : "name"},inplace=True)
avg_file_2_df.head()  
    

,name,avg1218
0,Alaska,6.817857
1,Arizona,6.272619
2,Arkansas,5.301190
3,California,6.795238
4,Colorado,4.695238


In [37]:
df = pd.read_csv("states.csv",delimiter="\t")
df.head()

,state,latitude,longitude,name
0,AK,63.588753,-154.493062,Alaska
1,AL,32.318231,-86.902298,Alabama
2,AR,35.201050,-91.831833,Arkansas
3,AZ,34.048928,-111.093731,Arizona
4,CA,36.778261,-119.417932,California


In [38]:
statelo_df=df
statelo_df.head()

,state,latitude,longitude,name
0,AK,63.588753,-154.493062,Alaska
1,AL,32.318231,-86.902298,Alabama
2,AR,35.201050,-91.831833,Arkansas
3,AZ,34.048928,-111.093731,Arizona
4,CA,36.778261,-119.417932,California


In [41]:
merge_df_2 = pd.merge(statelo_df, avg_file_2_df, on="name")
merge_df_2.head(10)

,state,latitude,longitude,name,avg1218
0,AK,63.588753,-154.493062,Alaska,6.817857
1,AR,35.201050,-91.831833,Arkansas,5.301190
2,AZ,34.048928,-111.093731,Arizona,6.272619
3,CA,36.778261,-119.417932,California,6.795238
4,CO,39.550051,-105.782067,Colorado,4.695238
5,CT,41.603221,-73.087749,Connecticut,6.076190
6,DE,38.910832,-75.527670,Delaware,5.321429
7,FL,27.664827,-81.515754,Florida,5.725000
8,GA,32.157435,-82.907123,Georgia,6.357143
9,HI,19.898682,-155.665857,Hawaii,3.816667


In [55]:
# gmaps.configure(api_key=g_key)
# locations = cities_df[["Lat", "Lng"]]
# Humidity = cities_df["Humidity"]
# max_humidity = cities_df["Humidity"].max()
import gmaps
from test_key import g_key
import os

gmaps.configure(api_key=g_key)
locations = merge_df_2[["latitude","longitude"]]
unemploymentrate2= merge_df_2[" avg1218"]
max_unemployment2=merge_df_2[" avg1218"].max()

In [56]:
# fig = gmaps.figure()
# heatmap = gmaps.heatmap_layer(locations, weights=Humidity, max_intensity=max_humidity)
# fig.add_layer(heatmap)

# fig
fig4 = gmaps.figure()
heatmap = gmaps.heatmap_layer(locations, weights = unemploymentrate2, max_intensity = max_unemployment2, point_radius=30)
fig4.add_layer(heatmap)

fig4

Figure(layout=FigureLayout(height='420px'))

In [48]:
rate_df_2=pd.read_csv("Resources/state_avg_1218.csv")
rate_df_2.head()

,State,avg1218
0,Alaska,6.817857
1,Arizona,6.272619
2,Arkansas,5.301190
3,California,6.795238
4,Colorado,4.695238


In [49]:
# rates = merge_df[" avg"]
# rate_info=[f"Average Unmployment Rate Avg:{rate}%" for rate in rates]
rates_2 = merge_df_2[" avg1218"]
rate_info_2 = [f"Average Unemployment Rate Avg 12-18:{rate}%" for rate in rates_2]
rate_info_2

['Average Unemployment Rate Avg 12-18:6.817857142857143%',
 'Average Unemployment Rate Avg 12-18:5.3011904761904765%',
 'Average Unemployment Rate Avg 12-18:6.272619047619048%',
 'Average Unemployment Rate Avg 12-18:6.795238095238097%',
 'Average Unemployment Rate Avg 12-18:4.695238095238095%',
 'Average Unemployment Rate Avg 12-18:6.076190476190478%',
 'Average Unemployment Rate Avg 12-18:5.321428571428571%',
 'Average Unemployment Rate Avg 12-18:5.7250000000000005%',
 'Average Unemployment Rate Avg 12-18:6.357142857142856%',
 'Average Unemployment Rate Avg 12-18:3.8166666666666655%',
 'Average Unemployment Rate Avg 12-18:3.875%',
 'Average Unemployment Rate Avg 12-18:4.592857142857143%',
 'Average Unemployment Rate Avg 12-18:6.595238095238094%',
 'Average Unemployment Rate Avg 12-18:5.463095238095237%',
 'Average Unemployment Rate Avg 12-18:4.3892857142857125%',
 'Average Unemployment Rate Avg 12-18:6.038095238095237%',
 'Average Unemployment Rate Avg 12-18:6.073809523809524%',
 'Ave

In [51]:
print(len(rate_info_2))
print(len(locations))
print(len(merge_df_2))

50
50
50


In [57]:
merge_df_2.head()

,state,latitude,longitude,name,avg1218
0,AK,63.588753,-154.493062,Alaska,6.817857
1,AR,35.201050,-91.831833,Arkansas,5.301190
2,AZ,34.048928,-111.093731,Arizona,6.272619
3,CA,36.778261,-119.417932,California,6.795238
4,CO,39.550051,-105.782067,Colorado,4.695238


In [ ]:
# fig3= gmaps.figure()
# heatmap = gmaps.heatmap_layer(locations, weights = unemploymentrate, max_intensity = max_unemployment, point_radius = 30)
# fig4.add_layer(heatmap)

# #demo_text = [f"{r['state']}: {r['rate']}" for i, r in demo_df.iterrows()]
# hover_text=[f"{r['name']}:{r[' avg']}" for i , r in merge_df.iterrows()]

# symbol_layer = gmaps.symbol_layer(merge_df[["latitude", "longitude"]], fill_color="blue",stroke_color="blue",scale= 2,
#                                 info_box_content= rate_info, hover_text=hover_text)

# fig3.add_layer(symbol_layer)
# fig3 

In [58]:
fig5= gmaps.figure()
heatmap = gmaps.heatmap_layer(locations, weights = unemploymentrate2, max_intensity = max_unemployment2, point_radius = 30 )
fig5.add_layer(heatmap)

hover_text_2 =[f"{r['name']}:{r[' avg1218']}" for i,r in merge_df_2.iterrows()]

symbol_layer = gmaps.symbol_layer(merge_df_2[["latitude", "longitude"]], fill_color = "purple", stroke_color="purple", 
                                 scale=2, info_box_content = rate_info_2, hover_text = hover_text_2)

fig5.add_layer(symbol_layer)
fig5

Figure(layout=FigureLayout(height='420px'))

In [ ]:
ls

In [ ]:
import time
from scipy.stats import linregress
from statepy import statepy

## Unemployment level

In [ ]:
# Pull unemploymend level data from Bureau of Labor Statistics

url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
headers = {"content-type": "application/json"}

target_series = ["LNS13000000", "LNS13023621", "LNS13023653", "LNS13025699", "LNS13023705", "LNS13023557", "LNS13023569"]




parameters = json.dumps({"seriesid": target_series,
             "startyear": start_year,
             "endyear": end_year,
             "registrationkey": bls_api_key})

p = requests.post(url, data=parameters, headers=headers)

json_data4 = json.loads(p.text)

json_data4

In [ ]:
years = []
months = []
date = []

for data_point in np.arange(len(json_data4["Results"]["series"][0]["data"])):
        years.append(json_data4["Results"]["series"][0]["data"][data_point]["year"])
        months.append(json_data4["Results"]["series"][0]["data"][data_point]["periodName"])
        date.append(f"{json_data['Results']['series'][0]['data'][data_point]['year']}-{json_data['Results']['series'][0]['data'][data_point]['period'][1:]}")

In [ ]:
# Create DataFrame

layoffs_df = pd.DataFrame({"Date": date,
                           "years": years,
                           "months": months,
                           "LNS13000000": "",
                           "LNS13023621": "",
                           "LNS13023653": "",
                           "LNS13025699": "",
                           "LNS13023705": "",
                           "LNS13023557": "",
                           "LNS13023569": ""})
layoffs_df.head()

In [ ]:
# Fills in values

for series4 in np.arange(len(json_data4["Results"]["series"])):
    for data_point4 in np.arange(len(json_data4["Results"]["series"][series4]["data"])):
        layoffs_df.loc[data_point4, json_data4["Results"]["series"][series4]["seriesID"]] = json_data4["Results"]["series"][series4]["data"][data_point4]["value"]


In [ ]:
# Rename column labels

layoffs_df.rename(columns={"LNS13000000": "Unemployment level, 16yo and over, thousands",
                           "LNS13023621": "Job losers, 16yo and over, thousands",
                           "LNS13023653": "Job losers on layoff, 16yo and over, thousands",
                           "LNS13025699": "Job losers not on layoff, 16yo and over, thousands",
                           "LNS13023705": "Job leavers (quit), 16yo and over, thousands",
                           "LNS13023557": "Reentrants to labor force, 16yo and over, thousands",
                           "LNS13023569": "New entrants to labor force, 16yo and over, thousands"},
                 inplace=True)

layoffs_df.sort_values(by="Date", ascending=True, inplace=True)

layoffs_df.to_csv("Unemployment_Levels.csv", index=False)

layoffs_df

## COVID cases

In [ ]:
url = "https://covidtracking.com/api/us/daily"

response = requests.get(url).json()

print(json.dumps(response, indent=True))

In [ ]:
dates = []
cum_positives = []
cum_deaths = []
inc_positives = []
inc_deaths = []

In [ ]:
for day in np.arange(len(response)):
    dates.append(response[day]["date"])
    cum_positives.append(response[day]["positive"])
    cum_deaths.append(response[day]["death"])
    inc_positives.append(response[day]["positiveIncrease"])
    inc_deaths.append(response[day]["deathIncrease"])

In [ ]:
sick_df = pd.DataFrame({"Date" : dates,
                       "Total Cases" : cum_positives,
                       "Total Deaths" : cum_deaths,
                       "Daily increase in Cases" : inc_positives,
                       "Daily increase in Deaths" : inc_deaths})

sick_df.loc[len(inc_positives)-1, "Daily increase in Deaths"] = sick_df.loc[len(cum_positives)-1, "Total Deaths"]
sick_df.loc[len(inc_deaths)-1, "Daily increase in Cases"] = sick_df.loc[len(cum_deaths)-1, "Total Cases"]

sick_df["Date"] = sick_df["Date"].astype(str)

In [ ]:
for x in np.arange(len(dates)):
    sick_df.loc[x,"Date"] = f"{sick_df.loc[x,'Date'][0:4]}-{sick_df.loc[x,'Date'][4:6]}-{sick_df.loc[x,'Date'][6:]}"

sick_df.sort_values(by="Date", ascending=True, inplace=True)

In [ ]:
sick_df.to_csv("COVID_cases.csv", index=False)
sick_df

## Economic Indicators

In [ ]:
file_path1 = "Resources/2020_GSPC.csv"
file_path2 = "Resources/2020_DJI.csv"
file_path3 = "Resources/2020_IXIC.csv"
file_path4 = "Resources/2020_RUT.csv"

SP500_df = pd.read_csv(file_path1)
Dow_df = pd.read_csv(file_path2)
NASDAQ_df = pd.read_csv(file_path3)
Russell2000_df = pd.read_csv(file_path4)

In [ ]:
SP500_df = SP500_df[["Date", "Close", "Volume"]]
Dow_df = Dow_df[["Date", "Close", "Volume"]]
NASDAQ_df = NASDAQ_df[["Date", "Close", "Volume"]]
Russell2000_df = Russell2000_df[["Date", "Close", "Volume"]]

In [ ]:
eco_ind_df = pd.merge(SP500_df, Dow_df, on="Date", how="outer", suffixes=["_SP500", "_Dow_Jones_Industrial_Average"])
eco_ind_2_df = pd.merge(NASDAQ_df, Russell2000_df, on="Date", how="outer", suffixes=["_NASDAQ", "_Russell2000"])
eco_ind_df = pd.merge(eco_ind_df, eco_ind_2_df, on="Date", how="outer")

eco_ind_df.sort_values(by="Date", ascending=True, inplace=True)

eco_ind_df

In [ ]:
eco_ind_df.to_csv("Eco_Indicators.csv", index=False)

In [ ]:
COVID_econ_df = pd.merge(sick_df, eco_ind_df, on="Date", how="inner")
COVID_econ_df = COVID_econ_df.drop(["Daily increase in Cases", "Daily increase in Deaths"], axis=1)
COVID_econ_df